# Projet Element Logiciel

In [1]:
import numpy as np
import pandas as pd
import random as rd
import copy
import time

## Data preparation

In [2]:
#On travaille sur un petit texte pour commencer
text  = open("data/TheBeatles.txt", "r") 
text = text.readlines()[0]

#Preprocessing : on supprime la ponctuation
not_alphabet="'?./§,;:!»«()…-" 
for i in not_alphabet:
    text = text.replace(i, "")
text = text.split(" ")
text_serie = pd.Series(text)
text_serie.head()

0     Après
1     avoir
2    débuté
3      sous
4        le
dtype: object

In [3]:
#one-hot encoding
from sklearn.preprocessing import LabelBinarizer
text_lb = LabelBinarizer()
X_hot = text_lb.fit_transform(text_serie.values)

In [4]:
X_hot

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

In [5]:
np.shape(X_hot)
#Les lignes sont chaque mot du texte dans l'ordre
#Les colonnes sont les différents mots
#ici 1300 mots dont 645 différents

(1300, 645)

In [6]:
X = np.apply_along_axis(np.argmax, 1, X_hot)

In [7]:
X
#Pour l'application on préfère définir chaque mot par un entier (ce qui est identique)

array([ 40, 171, 263, ..., 596,   0,   0], dtype=int64)

## Hogwild implementation

### in Python

In [8]:
#à modifier
def sig(x):
    return 1/(1+np.exp(-x))

def hog_loop(Min,Mout,alpha,wout,Nwin,negative):
    #Min matrice dans V*d
    #Mout matrice dans V*d
    #wout vecteur dans V*1 ou entier ? dans 0:V-1
    #nwin n vecteurs de V*1 (ie matrice de V*n) ou n entiers de 0:V-1
    V,d=Min.shape
    N=Nwin.shape[0]
    for i in range(N):
        input_word = Nwin[i]
        temp = np.array([0]*d)
        for k in range(negative+1):
            if k == 0:
                target_word = wout
                label = 1
            else : 
                target_word = rd.randint(0,V-1)
                label = 0
            inn = np.dot(Min[input_word,:],Mout[target_word,:])
            err = label - sig(inn)
            temp = err*Mout[target_word,:]
            Mout[target_word,:] = alpha*err*Min[input_word,:]
        Min[input_word,:]=alpha*temp
    return(Min,Mout)

In [9]:
#Un test
alpha=10**(-3)
Min = np.array([rd.random() for i in range(50*5)]).reshape((50,5))
Mout = np.array([rd.random() for i in range(50*5)]).reshape((50,5))
wout = 12
Nwin= np.array([1,8,27,4])
negative = 4
hog_loop(Min,Mout,alpha,wout,Nwin,negative)

(array([[ 5.61261596e-02,  5.09498761e-01,  4.44481245e-01,
          7.31734567e-01,  6.64943600e-01],
        [-3.72093841e-04, -4.14854147e-05, -3.01536942e-04,
         -4.35510735e-04, -1.22340393e-04],
        [ 5.59782209e-01,  3.68464106e-01,  4.54759580e-01,
          8.74249170e-01,  6.90428501e-01],
        [ 2.25292127e-01,  8.76824448e-01,  2.36051431e-01,
          1.02314982e-01,  7.28965088e-02],
        [ 1.35481184e-08,  2.58750512e-07,  3.37795720e-07,
          1.25186210e-07,  2.09784338e-07],
        [ 3.93491087e-02,  1.23549268e-01,  1.27318856e-01,
          7.98951919e-01,  2.96498680e-01],
        [ 3.72781607e-01,  8.70029721e-01,  9.09880425e-01,
          9.03354635e-01,  2.32403729e-01],
        [ 5.62360522e-01,  7.99426583e-01,  8.68153776e-02,
          1.92433797e-01,  8.38023593e-02],
        [-7.47730500e-05, -5.06962138e-04, -7.19844152e-04,
         -3.88831761e-04, -6.97081322e-05],
        [ 5.12021280e-01,  9.92012529e-01,  8.23149257e-01,
    

### in Pycuda

In [10]:
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

RMQ Dupre : 

regarder 
// ++i i++
   for(auto it: temp)
       *it = 0;
    
for(float* it = temp; it != ; ++it)
      *it = 0;
      
RMQ : peut-être pas la peine de passer random en entier à chaque loop

In [ ]:
mod = SourceModule("""

#include <time.h>
#include <stdlib.h>
#include <stdio.h>
#include <curand.h>
#include <math.h>



__device__ void loop(float *Min, float *Mout, float alpha, int *Nwin, int wout, int V, int d, int N, int negative, int* random, int idx){
    
    /*
    HOGWILD LOOP
    
    float *Min : initialisé en dehors de pycuda, poids de la PREMIERE couche du RN (attention float * et non float **) (V*d)
    float *Mout : initialisé en dehors de pycuda, poids de la DEUXIEME couche du RN (attention float * et non float **) (V*d)
    float alpha : learning rate
    int *Nwin : N input du contexte de l'output
    int wout : output
    int V : taille du vocabulaire (nombre de mots différents)
    int d : taille de l'espace de représentation
    int N : taille du contexte utilisé pour l'apprentissage (de chaque coté donc 2*N en tout)
    int negative : nb de negative utilisé pour l'apprentissage
    int* random : entiers aléat générés en dehors (difficulté pour générer de l'aléat sur le device) pour les negative
    int idx : indice de la parallélization (utile ?)
    
    Une boucle de l'Algo 1 de l'article. Permet de mettre à jour Min et Mout pour un wout (et son contexte associé).
    */

    /* Init variables */
    float* temp;
    temp = (float *)malloc(sizeof(float)*d);
    int label;
    int target_word;
    float inn;
    float err;
    
    /* Boucle principale sur les 2*N inputs*/
    for(int i=0;i<2*N;i++){
        int input_word = Nwin[i];
        for(int j=0;j<d;j++){
            temp[j]=0;
        }
        for(int k=0; k<negative+1;k++){
            if (k==0){
                target_word = wout;
                label = 1;
            } else {
                /* negative sampling */
                target_word = random[idx*negative+k-1];
                label = 0;
            }
            inn = 0;
            for(int j=0;j<d;j++){
                inn = inn + Min[input_word*d+j]*Mout[target_word*d+j];
            }
            err = label-(1/(1+exp(-inn)));
            for(int j=0;j<d;j++){
                temp[j] = temp[j]+err*Mout[target_word*d+j];
            }
            for(int j=0;j<d;j++){
                Mout[target_word*d+j] = Mout[target_word*d+j]+alpha*err*Min[input_word*d+j];
            }  
        }
        for(int j=0;j<d;j++){
            Min[input_word*d+j] = Min[input_word*d+j]+alpha*temp[j]; 
        }
    }
}


__global__ void parallel(int *X, float *Min, float* Mout, int* random, int* cst_int, float* cst_float) {

    /*
    Parallélisation
    int *X : la matrice du hot encoding
    float *Min : Min initialisé
    float *Mout : Mout initialisé
    int* random : entiers aléatoires générés hors du device
    int* cst_int : les constantes entières utiles (V,d,negative,N)
    float* cst_float : les constantes float utiles (alpha)
    */

    /*préparation des paramètres pour la fonction loop*/
    
    
    /*constantes entières passés depuis le code python*/
    int V = cst_int[0];
    int d = cst_int[1];
    int negative = cst_int[2];
    int N = cst_int[3];

    /*constante float passé depuis python*/
    float alpha = cst_float[0];
    
    /*l'index correspond au thread et indique simplement le wout sur lequel on travaille*/
    int index = threadIdx.x%(V-2*N)+N; /*+N pour éviter les effets de bord*/
    int wout = index;
    
    /*Calcul des mots du contexte de wout*/
    int *Nwin;
    Nwin = (int *)malloc(sizeof(int)*N*2);
    for(int i=0;i<N;i++){
        Nwin[i]= X[index+(i-N)];
    }
    for(int i=0;i<N;i++){
        Nwin[i+N]= X[index+1+i];
    }

    loop(Min,Mout,alpha,Nwin,wout,V,d,N,negative,random,index);
}
""")

### Tests

In [ ]:
#Initialisation des différentes variables
d = 10
V = np.shape(X_hot)[1]
negative = 5
N = 2
alpha = 0.001

cst_int = np.array([V,d,negative,N])
cst_float = np.array([alpha])

Min = np.random.rand(V,d)
Mout = np.random.rand(V,d)

random_neg = np.random.randint(V, size=(V,negative))

In [ ]:
Min = Min.astype(np.float32)
Min_orig = copy.deepcopy(Min)
Mout = Mout.astype(np.float32)
Mout_orig = copy.deepcopy(Mout)
random_neg = random_neg.astype(np.int32)
X = X.astype(np.int32)
cst_int = cst_int.astype(np.int32)
cst_float = cst_float.astype(np.float32)

Min_gpu = cuda.mem_alloc(Min.nbytes)
Mout_gpu = cuda.mem_alloc(Mout.nbytes)
random_neg_gpu = cuda.mem_alloc(random_neg.nbytes)
X_gpu = cuda.mem_alloc(X.nbytes)
cst_int_gpu = cuda.mem_alloc(cst_int.nbytes)
cst_float_gpu = cuda.mem_alloc(cst_float.nbytes)

cuda.memcpy_htod(Min_gpu, Min)
cuda.memcpy_htod(Mout_gpu, Mout)
cuda.memcpy_htod(random_neg_gpu, random_neg)
cuda.memcpy_htod(X_gpu,X)
cuda.memcpy_htod(cst_int_gpu,cst_int)
cuda.memcpy_htod(cst_float_gpu,cst_float)

In [ ]:
func = mod.get_function("parallel")

In [ ]:
t0=time.time()
func(X_gpu,Min_gpu,Mout_gpu,random_neg_gpu,cst_int_gpu,cst_float_gpu,block=(512,512,1))
dt=time.time()-t0
print("temps d'exécution",dt)

In [ ]:
cuda.memcpy_dtoh(Min,Min_gpu)
cuda.memcpy_dtoh(Mout,Mout_gpu)

In [ ]:
print(np.sum(Min!=Min_orig))
print(np.sum(Mout!=Mout_orig))

In [ ]:
"""10    : 0.00421"""
"""100   : 0.00507"""
"""1000  : 0.00400"""
"""see https://stackoverflow.com/questions/9985912/how-do-i-choose-grid-and-block-dimensions-for-cuda-kernels"""